In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score

# Входные данные #

In [3]:
train_df = pd.read_csv("./data/train.csv")
train_df.head()

,id,url,title,target
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False


Соотношение обычных и порнографических сайтов в обучающей выборке:

In [4]:
train_df['target'].value_counts()

False    118594
True      16715
Name: target, dtype: int64

In [5]:
test_df = pd.read_csv("./data/test.csv")
test_df.head()

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


# Самые простые соображения #
## Матрица признаков ##
В качестве текстов рассмотрим строки, получаемые объединением параметров **url** и **title**.

In [6]:
X = train_df["url"].values + " " + train_df["title"].values
y = train_df["target"].astype(int).values
print(X[:5])

['m.kp.md Экс-министр экономики Молдовы - главе МИДЭИ, цель которого сделать из республики не просителя, а донора: Надо избегать долгого нахождения н�'
 'www.kp.by Эта песня стала известна многим телезрителям благодаря сериалу Диверсант-2'
 'fanserials.tv Банши 4 сезон 2 серия Бремя красоты смотреть онлайн!'
 'colorbox.spb.ru Не Беси Меня Картинки'
 'tula-sport.ru В Новомосковске сыграют следж-хоккеисты алексинской «Звезды» и сборной Китая | Т...']


<br/>К полученному набору текстов применим модуль **CountVectorizer**

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

Давайте посмотрим на обучающую выборку:

In [7]:
vectorizer.fit(X)
vocabulary_len = len(vectorizer.get_feature_names())
print('Весь словарь содержит', vocabulary_len, 'слов')

porn_id = []
for id_ in train_df['id'].astype(int).values:
    if y[id_] == 1:
        porn_id.append(id_)

X_porn = X[porn_id]
X_porn_vectorized = vectorizer.fit_transform(X_porn)

vocabulary_len = len(vectorizer.get_feature_names())
print('Cловарь для сайтов с порнографией содержит', vocabulary_len, 'слов\n')

word_count = np.zeros(vocabulary_len)
for id_ in range(len(porn_id)):
    word_count += X_porn_vectorized.getrow(id_).toarray()[0]
    
dictionary = {}
for id_ in range(vocabulary_len):
    if word_count[id_] > 100:
        dictionary.update([(vectorizer.get_feature_names()[id_], int(word_count[id_]))])

sorted_dictionary = sorted(dictionary.items(), key = lambda i: i[1], reverse = True)

print("Наиболее часто встречающиеся слова на сайтах с порнографией:\n")
for i in sorted_dictionary[:20]:
    print(i[0], ':', i[1])

Весь словарь содержит 215740 слов
Cловарь для сайтов с порнографией содержит 28608 слов

Наиболее часто встречающиеся слова на сайтах с порнографией:

com : 7052
порно : 5207
porn : 4259
видео : 3055
net : 2902
sex : 2330
videos : 1971
ru : 1948
на : 1910
www : 1811
video : 1659
онлайн : 1624
hd : 1585
xxx : 1499
biqle : 1470
porno : 1431
daftsex : 1355
смотреть : 1312
free : 1257
секс : 1163


Для оценки качества модели используем перекресную проверку.

In [7]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5)

In [12]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()

Результат для **MultinomialNB**:

Train set (mean): 0.9255191562541937<br/>
Test set (mean):  0.9310207266931716

In [9]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C=1, max_iter = 1000, solver='lbfgs')

Результат для **LogisticRegression**:

Train set (mean): 0.9904633707568836<br/>
Test set (mean):  0.9655801283019537

In [18]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=5)

Результат для **DecisionTreeClassifier**:

Train set (mean): 0.735538626434425<br/>
Test set (mean):  0.7335053799679688

In [10]:
f1_mean_train = 0
f1_mean_test  = 0
for train, test in skf.split(X, y):
    X_train_vectorized = vectorizer.fit_transform(X[train])
    X_test_vectorized = vectorizer.transform(X[test])
    
    model.fit(X_train_vectorized, y[train])
    print('Train set:')
    y_pred = model.predict(X_train_vectorized)
    f1 = f1_score(y[train], y_pred)
    print(f1)
    f1_mean_train += f1
    
    print('Test set:')
    y_pred = model.predict(X_test_vectorized)
    f1 = f1_score(y[test], y_pred)
    print(f1)
    f1_mean_test += f1
    print('\n')
    
print('Train set (mean):\n', f1_mean_train/5)
print('Test set (mean):\n', f1_mean_test/5,'\n')

Train set:
0.9910650329877474
Test set:
0.9626543209876544


Train set:
0.9906838154867422
Test set:
0.9640820101741945


Train set:
0.9903388935013963
Test set:
0.9631457208943717


Train set:
0.9904955872369313
Test set:
0.9696136814780882


Train set:
0.989733524571601
Test set:
0.9684049079754601


Train set (mean):
 0.9904633707568836
Test set (mean):
 0.9655801283019537 



## Итоговая модель ##

In [11]:
X_train = train_df["url"].values + " " + train_df["title"].values 
X_test  = test_df["url"].values  + " " + test_df["title"].values 

y_train = train_df["target"].astype(int).values

X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

model = LogisticRegression(C=1, max_iter = 1000, solver='lbfgs')
model.fit(X_train_vectorized, y_train)
y_pred_train = model.predict(X_train_vectorized)
print('Train set:\n', f1_score(y_train, y_pred_train))

y_pred_test = model.predict(X_test_vectorized)
file = open('task2_output.txt', 'w')

import csv
i = 0
with open('task2_result.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(['id', 'target'])
    for id_ in test_df["id"].values:
        writer.writerow([id_, y_pred_test[i].astype(bool)])
        i = i + 1

Train set:
 0.9908898944193062


Где прогноз оказался неверным:

In [12]:
err_FP_count = 0
err_FN_count = 0
for id_ in train_df["id"].astype(int).values:
    if(y_pred_train[id_] < y_train[id_]):
        err_FN_count += 1
    if(y_pred_train[id_] > y_train[id_]):
        err_FP_count += 1

print("Expected 'True', predicted 'False':", err_FN_count, 'errors\n') 
count = 0
for id_ in train_df["id"].astype(int).values:  
    if(y_pred_train[id_] < y_train[id_] and count < 10):
        print(X_train[id_])
        count += 1

print("\nExpected 'False', predicted 'True':", err_FP_count, 'errors\n') 
count = 0
for id_ in train_df["id"].astype(int).values:  
    if(y_pred_train[id_] > y_train[id_] and count < 10):
        print(X_train[id_])
        count += 1


Expected 'True', predicted 'False': 291 errors

mail.ru Екатерина Коба голая - 191 тыс. результатов. Поиск Mail.Ru
filmpornobesplatno.com 年轻美女的阴部和肛门被振动器和嗡嗡声搞砸了
inseros.com Раздвинутые ножки Рамоны
fotosdemulheresnuas.org 成熟的宝贝的阴道享受别致的handjob到凸轮 | 胖性爱视频
pornososalka.com Результаты поиска по страпонит
dvdpornrip.com Caribbeancom download all high quality » page 2
vinceste.com Конча сына в ротике мамашки
i-borneo.fr gif animй sexe gratuit
mail.ru бесплатная жесткая порнуха онлайн - Поиск Mail.Ru
mail.ru красивой женщине кончили в рот - Поиск Mail.Ru

Expected 'False', predicted 'True': 11 errors

intimi.lv Intim.lv - Sludinājumu serviss
xn--e1afbimzh3a.cc Инцест с разговорами на Инцестик.cc - страница 4 из 82
18comix.com CrazyDad3D - Love Me Tender Part 9 » 18Comix - Free Adult Comics
minet.club туркменская секс изнасилование - минет клуб
privat-zapisi.co Профиль пользователя - privat-zapisi.com
mail.ru Секс Музыка для интимной Эротические моменты :: video.mail.ru
video-kursi.net Мастерск